In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
 -0.335344 
 -0.925595 
 -0.0433723
 -0.170107 
  0.578357 
  0.343653 
  0.57062  

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 3.1620974168503886 rad about [0.1853003577664265,0.5962663474562686,0.7811083281484039], translation: Vec(0.9481153259289057,0.528942665312469,0.43087776807738365)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(0.5067326619545522,0.14494474200443716,0.7397772203163864)

In [12]:
mass(mechanism)

1.3158049075210412

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(0.4351198055020581,0.5691110192877123,0.801042678133964)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.0740682659041797,-0.1481554879603154,0.5953683356749456), linear: Vec(0.4575095492235725,-0.1997247123363437,-0.106618453410956)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.0853042   0.863225    0.1107      …   0.0         0.0         0.0     
  0.17063    -0.0586856   0.991051       -0.0        -0.0        -0.0     
 -0.685684    0.501397   -0.0745882      -0.0        -0.0        -0.0     
 -0.526913    0.283417   -0.191566        0.863225    0.1107      0.492533
  0.230022    0.207977    0.00542881     -0.0586856   0.991051   -0.119891
  0.122792   -0.463599   -0.212179    …   0.501397   -0.0745882  -0.861996

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(0.6960835293758662,0.9822241228525848,-1.2504013566803684), linear: Vec(0.350219876667211,0.4722668940436466,-0.6799725589940665)

In [18]:
mass_matrix(x)

7x7 Array{Float64,2}:
  0.566883      0.321808      0.605006     …   0.486471      0.141419 
  0.321808      2.44611       0.00373972       1.16809       0.151564 
  0.605006      0.00373972    2.40962         -1.11022e-16   0.542572 
 -2.77556e-17   0.175543     -0.486471         5.55112e-16  -0.0313834
 -0.175543     -1.38778e-17  -1.16809         -9.71445e-17  -0.230954 
  0.486471      1.16809      -2.77556e-17  …   1.3158        0.0667275
  0.141419      0.151564      0.542572         0.0667275     0.173292 

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -0.635889  -0.0702468  -1.79538   …   0.978917    0.047697  -0.411165 
 -0.184479  -2.37489     0.428206     -0.157488   -1.05418   -0.0652796
  0.346287   0.399244    1.30064      -0.639682    0.173874   0.361309 
 -0.220171  -0.726854    0.549241     -0.145659   -0.648078   0.019792 
  0.232295   0.150345    1.12908      -1.30403     0.157753   0.235045 
  0.406243   0.918868    0.15679   …   0.0981435   1.13422    0.056028 

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(-1.828435742298753,-3.4470007882241283,1.9707258818051654), linear: Vec(-1.982441048220758,1.0318919619965072,1.4928906191969427)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.02600620819440716,-0.004082563761851031,0.8266374758505901), linear: Vec(0.698204953425547,-0.3585915332427634,1.2950752147834457)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.3995313181373058,0.5070411525488001,0.17588023246476486), linear: Vec(0.17402817972568452,0.7567294250952441,0.6752689849562643)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

Dict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "joint1": Quatern… => [4.222393132944168,10.144730735450448,1.073765877…
  Joint "joint2": Revolut… => [0.5199014791701626]

In [24]:
relative_acceleration(x, body1, body2, v̇)

SpatialAcceleration of "after_joint1" w.r.t "after_joint2" in "world":
angular: Vec(0.3372817413425344,-0.38252606754217156,-0.6762487300979925), linear: Vec(-0.55853530259498,-0.5888737494341066,-0.5856630852862068)

In [25]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [26]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
    Vertex: body3, Edge: joint3
      Vertex: body4, Edge: joint4
        Vertex: body8, Edge: joint8
          Vertex: body10, Edge: joint10
    Vertex: body7, Edge: joint7
  Vertex: body5, Edge: joint5
    Vertex: body6, Edge: joint6
    Vertex: body9, Edge: joint9
  Vertex: body11, Edge: joint11